In [14]:
import math
import random
import logging
from datetime import datetime

import torch
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import ImageGrid

QUANTITY_OF_TEST_CASES = 20
LOGGING_LEVEL = logging.NOTSET # logging.DEBUG

In [15]:
class CustomFormatter(logging.Formatter):

    cyan = "\x1b[36;20m"
    blue = "\x1b[34;20m"
    grey = "\x1b[38;20m"
    yellow = "\x1b[33;20m"
    red = "\x1b[31;20m"
    bold_red = "\x1b[31;1m"
    reset = "\x1b[0m"
    format = "[%(asctime)s - %(levelname)s]: %(message)s (%(filename)s:%(lineno)d)"

    FORMATS = {
        logging.DEBUG: cyan + format + reset,
        logging.INFO: blue + format + reset,
        logging.WARNING: yellow + format + reset,
        logging.ERROR: red + format + reset,
        logging.CRITICAL: bold_red + format + reset
    }

    def format(self, record):
        log_fmt = self.FORMATS.get(record.levelno)
        formatter = logging.Formatter(log_fmt)
        return formatter.format(record)

logger = logging.getLogger("notebook")
logger.setLevel(LOGGING_LEVEL)

ch = logging.StreamHandler()
ch.setLevel(LOGGING_LEVEL)
ch.setFormatter(CustomFormatter())
logger.addHandler(ch)

In [16]:
def params_to_string(params: dict) -> str:
    name = ""
    for key, value in params.items():
        name += f"{key}_{value}-"
    return name[:-1]

In [17]:
class RotationAndFlipLayer(torch.nn.Module):
    """
    A layer that converts a (B, L, C, W, H) into a (B * 8 * L, C, W, H)
    """
    def __init__(self, rot: bool = True):
        super().__init__()
        self.metadata = None
        self.rot = rot
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch, levels, channels, width, height = x.shape
        self.metadata = { "batch": batch, "levels": levels, "transforms": 8 }
        
        if not self.rot:
            transforms = (
                x,
                x,
                x,
                x,
                x, 
                x,
                x,
                x,
            )
        else:
            width_dim = len(x.shape) - 2
            height_dim = len(x.shape) - 1
            
            flipped: torch.Tensor = x.flip(dims=(height_dim,))
            flipped_rot90: torch.Tensor = flipped.rot90(k=1, dims=(width_dim, height_dim))
            flipped_rot180: torch.Tensor = flipped.rot90(k=2, dims=(width_dim, height_dim))
            flipped_rot270: torch.Tensor = flipped.rot90(k=3, dims=(width_dim, height_dim))
            rot90: torch.Tensor = x.rot90(k=1, dims=(width_dim, height_dim))
            rot180: torch.Tensor = x.rot90(k=2, dims=(width_dim, height_dim))
            rot270: torch.tensor = x.rot90(k=3, dims=(width_dim, height_dim))
    
            transforms = (
                x,
                rot90,
                rot180,
                rot270,
                flipped, 
                flipped_rot90,
                flipped_rot180,
                flipped_rot270,
            )

        x = torch.cat(transforms, dim=1)
        x = x.reshape(batch * levels * len(transforms), channels, width, height)

        self.metadata = { "batch": batch, "levels": levels, "transforms": 8 }
        return x

def test_rotation_and_flip_layer():
    layer = RotationAndFlipLayer()
    
    for _ in range(QUANTITY_OF_TEST_CASES):
        batch = random.randint(1, 100)
        levels = random.randint(1, 100)

        x = torch.zeros((batch, levels, 1, 30, 30))
        out = layer.forward(x)

        transforms = layer.metadata["transforms"]

        expected = batch * levels * 8
        assert out.shape == torch.Size([expected, 1, 30, 30]), f"Test failed with batch {batch} and level {levels}"

        del x
        del out

    del layer

test_rotation_and_flip_layer()

In [18]:
class ImageVisualizer:
    def __init__(self):
        self._samples_array: list[tensor.Tensor] = []
        self._levels = 5

    @property
    def nsamples(self) -> int:
        return len(self._samples_array)
    
    def plot(self) -> None:
        fig = plt.figure(figsize=(14., 14.))
        grid = ImageGrid(fig, 111, nrows_ncols=(int(self.nsamples / self._levels), self._levels), axes_pad=0.2)
        for ax, im in zip(grid, self._samples_array):
            ax.imshow(im)

        for level in range(1, self._levels + 1):
            grid.axes_all[level - 1].set_title(f"Level {level}")
        plt.show()
        
    def add(self, images: torch.Tensor) -> None:
        if len(images.shape) != 4:
            logger.error("Expected 4D tensor: (levels, channels, width, height), received: %s", images.shape)
            raise ValueError(f"Expected 4D tensor: (levels, channels, width, height), recieved: {images.shape}")

        if images.shape[0] != self._levels:
            logger.error("Expected 5 levels, received:  %s", images.shape[0])
            raise ValueError(f"Expected 5 levels, received: {images.shape[0]}")
        
        self._samples_array.extend([im.squeeze().numpy().copy() for im in images.chunk(self._levels)])

    def clean(self) -> None:
        self._samples_array = []

def test_image_visualizer():
    t1 = torch.rand(5, 1, 30, 30)
    t2 = t1.flip(dims=(2,))
    t3 = t1.rot90(dims=(2, 3))

    visualizer = ImageVisualizer()
    visualizer.add(t1)
    visualizer.add(t2)
    visualizer.add(t3)
    
    assert visualizer.nsamples == 15

    del visualizer
    del t3
    del t2
    del t1

test_image_visualizer()

In [19]:
class DELIGHTModel(torch.nn.Module):
    """
    DELIGHT implementation written in torch.

    Allows inputs of ([B]atch, [L]evel, 1, 30, 30).

    """
    def __init__(self,
                 rot: bool = True,
                 levels: int = 5, 
                 nconv1: int = 52, 
                 nconv2: int = 57, 
                 nconv3: int = 41, 
                 ndense: int = 685
    ):
        super().__init__()
        self.LEVELS = levels
        self.CONV2D_ONE_OUT_CHANNELS = nconv1
        self.CONV2D_TWO_OUT_CHANNELS = nconv2
        self.CONV2D_THREE_OUT_CHANNELS = nconv3
        self.LINEAR_TWO_IN = ndense

        h = w = 30
        conv_kernel_size = 3
        mp_kernel_size = 2

        self.rot_and_flip = RotationAndFlipLayer(rot=rot)
        self.conv1 = torch.nn.Conv2d(
            in_channels=1,
            out_channels=self.CONV2D_ONE_OUT_CHANNELS, 
            kernel_size=conv_kernel_size
        )
        
        self.conv2 = torch.nn.Conv2d(
            in_channels=self.conv1.out_channels, 
            out_channels=self.CONV2D_TWO_OUT_CHANNELS,
            kernel_size=conv_kernel_size
        )
        
        self.conv3 = torch.nn.Conv2d(
            in_channels=self.conv2.out_channels, 
            out_channels=self.CONV2D_THREE_OUT_CHANNELS, 
            kernel_size=conv_kernel_size
        )
        
        self.relu = torch.nn.ReLU()
        self.max_pool = torch.nn.MaxPool2d(kernel_size=mp_kernel_size)
        self.flatten = torch.nn.Flatten()
        
        self.bottleneck = torch.nn.Sequential(
            self.conv1,
            self.relu,
            self.max_pool, 
            self.conv2,
            self.relu,
            self.max_pool,
            self.conv3,
            self.relu,
            self.flatten,
        )

        linear_in = self._compute_fc1_features(
            width=w,
            height=h,
            conv_kernel_size=conv_kernel_size,
            mp_kernel_size=mp_kernel_size
        )

        self.fc1 = torch.nn.Linear(in_features=linear_in, out_features=self.LINEAR_TWO_IN)
        self.fc2 = torch.nn.Linear(in_features=self.LINEAR_TWO_IN, out_features=2)
        self.tanh = torch.nn.Tanh()

    def _compute_fc1_features(self, *, width: int, height: int, conv_kernel_size: int, mp_kernel_size: int) -> int:
        height = height - conv_kernel_size + 1          # conv2d 1 
        height = math.floor((height - mp_kernel_size)/2 + 1)  # maxpool2d 1 
        height = height - conv_kernel_size + 1          # conv2d 2
        height = math.floor((height - mp_kernel_size)/2 + 1)  # maxpool2d 2
        height = height - conv_kernel_size + 1          # conv2d 3

        width = width - conv_kernel_size + 1          # conv2d 1
        width = math.floor((width - mp_kernel_size)/2 + 1)  # maxpool2d 1
        width = width - conv_kernel_size + 1          # conv2d 2
        width = math.floor((width - mp_kernel_size)/2 + 1)  # maxpool2d 2
        width = width - conv_kernel_size + 1          # conv2d 3
        return height * width * self.CONV2D_THREE_OUT_CHANNELS * self.LEVELS
                
    def forward(self, x: torch.Tensor) -> torch.Tensor:                
        # Apply flips and rotations over level (L) dimension
        logger.debug("Received input x with shape %s", x.shape)
        x = self.rot_and_flip(x)
        batch_size = self.rot_and_flip.metadata["batch"]
        n_transforms = self.rot_and_flip.metadata["transforms"]
        logger.debug("Input was transformed into %s rotations and flip, ended with shape %s", n_transforms, x.shape) 
        
        # Bottleneck
        x = self.bottleneck(x)
        logger.debug("Input shape after bottleneck: %s", x.shape)
        
        # Undo transformations
        x = x.reshape(batch_size, n_transforms, -1)
        logger.debug("Input shape after reshape: %s", x.shape)
        
        # Linear
        x = self.fc1(x)
        logger.debug("Input shape after first fully-connected layer: %s", x.shape)
        x = self.tanh(x)
        x = self.fc2(x)
        logger.debug("Input shape after last fully-connected layer: %s", x.shape)
        x = x.reshape((batch_size, n_transforms * 2))
        logger.debug("Output shape: %s", x.shape)
        
        return x
def test_input_output_parameters_on_delight_model():
    for _ in range(QUANTITY_OF_TEST_CASES):
        batch = random.randint(1, 32)
        levels = random.randint(1, 5)

        x = torch.zeros((batch, levels, 1, 30, 30))
        model = DELIGHTModel(levels=levels)

        try:
            out = model.forward(x)
        except RuntimeError:
            logger.error("Runtime error on model with batch %s and levels %s", batch, levels)
            assert False, f"Runtime error on model with batch {batch} and levels {levels}"
        
        expected = torch.Size([batch, 16])
        assert out.shape == expected, f"Failed with batch {batch} and levels {levels} => {out.shape} != {expected}"
        del out
        del model
        del x

test_input_output_parameters_on_delight_model()

In [20]:
import os
from enum import Enum
from typing import Union
from dataclasses import dataclass

import numpy as np
from torch.utils.data import Dataset

class CustomDatasetType(Enum):
    TRAIN = "TRAIN"
    TEST = "TEST"
    VALIDATION = "VALIDATION"
    
@dataclass
class CustomDatasetOptions:
    dataset_type: CustomDatasetType
    n_levels: int
    fold: int
    mask: bool
    object: bool

    def get_filenames(self) -> str:
        if self.dataset_type == CustomDatasetType.TRAIN:
            X = "X_train_nlevels%i_fold%i_mask%s_objects%s.npy" % (self.n_levels, self.fold, self.mask, self.object)
            y = "y_train_nlevels%i_fold%i_mask%s_objects%s.npy" % (self.n_levels, self.fold, self.mask, self.object)
        elif self.dataset_type == CustomDatasetType.TEST:
            X = "X_test_nlevels%i_mask%s_objects%s.npy" % (self.n_levels, self.mask, self.object)
            y = "y_test_nlevels%i_mask%s_objects%s.npy" % (self.n_levels, self.mask, self.object)
        else:
            X = "X_val_nlevels%i_fold%i_mask%s_objects%s.npy" % (self.n_levels, self.fold, self.mask, self.object)
            y = "y_val_nlevels%i_fold%i_mask%s_objects%s.npy" % (self.n_levels, self.fold, self.mask, self.object)
            
        return X, y
            
class CustomDataset(Dataset):    
    def __init__(self, options: CustomDatasetOptions, source: Union[str, None] = None, rot: bool = True):
        self.source = source if source is not None else "/home/fforster/SNHosts/data"
        X_path, y_path = options.get_filenames()
        self.X = torch.Tensor(np.load(os.path.join(self.source, X_path))).permute(0, 3, 1, 2)
        self.y = torch.Tensor(self.rotateY(np.load(os.path.join(self.source, y_path)), rot))

    def rotateY(self, y: np.ndarray, rot: bool) -> np.ndarray:
        if not rot:
            return np.concatenate([
                y,
                y,
                y,
                y,
                y,
                y,
                y,
                y
            ], axis=1)
            
        y90 = [-1, 1] * y[:, ::-1]
        y180 = [-1, 1] * y90[:, ::-1]
        y270 = [-1, 1] * y180[:, ::-1]
        yflip = [1, -1] * y
        yflip90 = [-1, 1] * yflip[:, ::-1]
        yflip180 = [-1, 1] * yflip90[:, ::-1]
        yflip270 = [-1, 1] * yflip180[:, ::-1]

        return np.concatenate([
            y,
            y90,
            y180,
            y270,
            yflip,
            yflip90,
            yflip180,
            yflip270
        ], axis=1)


    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx: int):
        X = self.X[idx]
        y = self.y[idx]

        if len(X.shape) == 3: # has no channel information
            levels, width, height = X.shape 
            X = X.reshape(levels, 1, width, height) # asume 1 channel information
        return X, y

In [21]:
train_opt = CustomDatasetOptions(
    dataset_type=CustomDatasetType.TRAIN,
    n_levels=5,
    fold=0,
    mask=False,
    object=True
)
test_opt = CustomDatasetOptions(
    dataset_type=CustomDatasetType.TEST,
    n_levels=5,
    fold=0,
    mask=False,
    object=True
)
val_opt = CustomDatasetOptions(
    dataset_type=CustomDatasetType.VALIDATION,
    n_levels=5,
    fold=0,
    mask=False,
    object=True
)

batch_size = 32
source = "/home/keviinplz/universidad/tesis/snhost/data"

In [22]:
from torch.utils.data import DataLoader

params = {
    "rot": True,
    "levels": 5,
    "nconv1": 16,
    "nconv2": 32,
    "nconv3": 32,
    "ndense": 128
}

train = CustomDataset(options=train_opt, source=source, rot=params["rot"])
test = CustomDataset(options=test_opt, source=source, rot=params["rot"])
val = CustomDataset(options=val_opt, source=source, rot=params["rot"])
train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val, batch_size=batch_size, shuffle=True)
model = DELIGHTModel(**params)

In [23]:
%load_ext tensorboard
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-4)

name = params_to_string(params)
writer = SummaryWriter('runs/delight_{}_{}'.format(name, ts))

EPOCHS = 50

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
def train_one_epoch(epoch: int, tb_writer: SummaryWriter, device: str = "cuda"):
        running_loss = 0.
        last_loss = 0.
    
        # Here, we use enumerate(training_loader) instead of
        # iter(training_loader) so that we can track the batch
        # index and do some intra-epoch reporting
        pbar = tqdm(train_dl, leave=False, position=1)
        for i, data in enumerate(pbar):
            # Every data instance is an input + label pair
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
    
            # Zero your gradients for every batch!
            optimizer.zero_grad()
    
            # Make predictions for this batch
            outputs = model(inputs)
    
            # Compute the loss and its gradients
            loss = loss_fn(outputs, labels)
            loss.backward()
    
            # Adjust learning weights
            optimizer.step()
    
            # Gather data and report
            running_loss += loss.item()
            if i % batch_size == batch_size - 1:
                last_loss = running_loss / batch_size # loss per batch
                pbar.set_description('batch {} loss: {}'.format(i + 1, last_loss))
                tb_x = epoch * len(train_dl) + i + 1
                tb_writer.add_scalar('Loss/train', last_loss, tb_x)
                running_loss = 0.
    
        return last_loss

In [25]:
%tensorboard --logdir runs

best_vloss = 1_000_000.
device = "cuda"

os.makedirs("states", exist_ok=True)

pbar = tqdm(range(EPOCHS), leave=False, position=0)

for epoch in pbar:
    pbar.set_description("Running epoch %s" % epoch)

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    model.to(device)
    avg_loss = train_one_epoch(epoch, writer, device)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_dl):
            vinputs, vlabels = vdata
            vinputs = vinputs.to(device)
            vlabels = vlabels.to(device)
            
            voutputs = model(vinputs)

            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    pbar.set_description("LOSS train %s valid %s" % (avg_loss, avg_vloss), refresh=False)

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'states/model_{}_{}'.format(ts, epoch)
        torch.save(model.state_dict(), model_path)

    epoch += 1

Reusing TensorBoard on port 6006 (pid 87298), started 2:37:18 ago. (Use '!kill 87298' to kill it.)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

...

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

In [26]:
best_vloss

tensor(51.7013, device='cuda:0')